In [ ]:
# FAISS를 활용하는 과정을 코드 상에서 이해하고 Passage Retrieval를 수행

# 필요 패키지 설치

In [ ]:
!pip install datasets
!pip install transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm import tqdm, trange
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)

torch.manual_seed(810)
torch.cuda.manual_seed(810)

# Dense embedding을 활용한 passage / question enocoder 학습

 Dense Embedding을 통해 Passage와 Question vector을 얻어야함

이를 위해 passage / question enocoder 학습

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

# Training Dataset 준비

In [ ]:
# Use subset (10 example) of original training dataset
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]

# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break
q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')

max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

# BERT encoder 학습

In [ ]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()

  def forward(self, input_ids,
              attention_mask=None, token_type_ids=None):

      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)

      pooled_output = outputs[1]

      return pooled_output

def train(args, num_neg, dataset, p_model, q_model):

# Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

# Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

# Start training!
  global_step = 0

  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()

  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()

      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }

      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}

      p_outputs = p_model(**p_inputs)#(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)#(batch_size*, emb_dim)# Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()#(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1

      torch.cuda.empty_cache()

  return p_model, q_model


In [ ]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# Train function 정의 후, 두개의 encoder fine-tuning 하기

p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

tensor(6.4907, device='cuda:0', grad_fn=<NllLossBackward0>)



Iteration:  10%|█         | 1/10 [00:03<00:28,  3.22s/it]

tensor(3.3085, device='cuda:0', grad_fn=<NllLossBackward0>)



Iteration:  20%|██        | 2/10 [00:04<00:16,  2.00s/it]

...


Iteration:  80%|████████  | 8/10 [00:08<00:02,  1.03s/it]

tensor(5.5848e-05, device='cuda:0', grad_fn=<NllLossBackward0>)



Iteration:  90%|█████████ | 9/10 [00:09<00:01,  1.03s/it]

tensor(1.8298e-05, device='cuda:0', grad_fn=<NllLossBackward0>)



Epoch: 100%|██████████| 2/2 [00:23<00:00, 11.66s/it]


# Passage retrieval 준비

Training 데이터셋을 통해 각각의 Encoder을 학습시켰기 때문에 이를 활용해 Passage Retrieval을 진행

In [ ]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

# Passage encoder를 활용하여 passage dense embedding 생성

In [ ]:
eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }

    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

p_embs = np.array(p_embs)
p_embs.shape # (num_passage, emb_dim)

Iteration: 100%|██████████| 120/120 [00:26<00:00,  4.55it/s]


(960, 768)

# Question encoder를 활용해여 quesntion dense embedding 생성

In [ ]:
sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['1990년대 중후반 광고 모델로 활약하며 모든 패션과 유행의 아이콘이었던 연예인은?',
 '음성 반송파는 스테레오 음성과 외국어 방송 또는 어떤 방송에 사용되는가?',
 '5월 7일 김대중이 총선 기간 동안 맡은 자리는?',
 '방탄소년단이 미국 빌보드 200에 첫 진입한 해는?',
 '케라톱스 호리두스를 명명한 사람은 누구인가?']

In [ ]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape# (num_query, emb_dim)

(5, 768)

# GPU를 활용하여 passage retrieval 수행

GPU에서 exhaustive search 수행

In [ ]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [ ]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[733, 158,  36,  ..., 218, 592, 169],
        [733, 158, 599,  ..., 938, 546, 956],
        [733, 158,  36,  ..., 218, 592, 169],
        [733, 158,  36,  ..., 592, 956, 546],
        [733, 158, 599,  ..., 546, 938, 956]], device='cuda:0')
--- 0.013309955596923828 seconds ---


In [ ]:
k = 5

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]], '\n')
    # print('\n')


[Search query]
 1990년대 중후반 광고 모델로 활약하며 모든 패션과 유행의 아이콘이었던 연예인은? 

[Ground truth passage]
김희선은 1990년대 중후반 그녀만의 통통 튀고 발랄한 신세대 이미지로 매력을 뽐내며 인기를 끌기 시작했고, 드라마 《목욕탕집 남자들》(1995)에 이어 《웨딩드레스》, 《프로포즈》(1997), 《세상 끝까지》, 《미스터Q》(1998), 《토마토》(1999), 《해바라기》(1999) 등의 높은 시청률을 기록한 인기 드라마에서 연거푸 주연을 맡았고, 이외에도 수십 편의 광고 모델로 활약하며 명실상부 최고의 인기 톱스타로 자리매김했으며 그 당시 모든 패션과 유행의 아이콘이었다. 또한 김희선 황신혜의 뒤를 이은 1990년대를 대표하는 미인으로 불리며, 또한 당시 만 22세의 나이로 드라마 《미스터Q》로 SBS 연기대상에서 최연소의 나이에 연기대상을 수상하였고, 국내 외에도 드라마의 폭발적인 인기에 힘입어 중국에서도 대표적인 한류 스타로 발돋음하였다. 하지만 많은 인기를 끈 드라마와는 달리 영화에서는 《패자부활전》(1997년), 《카라》, 《자귀모》(1999년), 《비천무》(2000년)가 연이어 흥행참패를 당하면서 연기력 논란에 휩싸이기도 했다. 

Top-1 passage with score 35.9678
김희선(1977년 6월 11일 ~ )은 대한민국의 배우이다. 경상북도 대구 출생한 김희선은 1992년 '고운 얼굴 선발대회'에서 대상 입상을 계기로 틴잡지 모델로 활동하였고, 1993년 롯데삼강 '꽃게랑' 광고로 연예계에 데뷔했다. 데뷔 이후 90년대 후반을 대표하는 톱스타로 자리매김하여, 1990년대 후반의 텔레비전 드라마 《목욕탕집 남자들》(1995), 《프로포즈》(1997), 《미스터큐》, 《세상끝까지》(1998),《토마토》,《해바라기》(1999)와 영화 《비천무》(2000년), 《와니와 준하》(2001) 등의 작품으로 많은 사랑을 받았다. 2012년 퓨전 사극 《신의》로 6년 만에 브라운관에 컴백한 후

# FAISS를 활용하여 CPU에서 passage retrieval 수행

FAISS 는 유사도 탐색을 위한 라이브러리

FAISS의 기능인 SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습

위의 Brute-force 방식과 어떤 차이가 있는 지 확인

In [ ]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

In [ ]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [ ]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))
print()
print('=======[Distance]=======')
print(D)
print()
print('=======[Index of Top-5 Passages]=======')
print(I)

--- 0.0041522979736328125 seconds ---

=======[Distance]=======
[[25.136074 25.861015 27.076565 27.149023 27.193035]
 [29.5696   31.213871 33.32358  34.764877 36.734577]
 [26.286247 26.42641  26.706371 27.406494 29.237589]
 [29.591427 31.638199 32.28366  34.1876   34.736015]
 [26.680216 27.855738 30.239388 31.825064 34.64506 ]]

=======[Index of Top-5 Passages]=======
[[599 211 354  36 488]
 [488 208 211 354 599]
 [354 211 488 599  36]
 [488 208 211 354 599]
 [488 208 211 354 599]]


In [ ]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
    print()

[Search query]
 1990년대 중후반 광고 모델로 활약하며 모든 패션과 유행의 아이콘이었던 연예인은? 

[Ground truth passage]
김희선은 1990년대 중후반 그녀만의 통통 튀고 발랄한 신세대 이미지로 매력을 뽐내며 인기를 끌기 시작했고, 드라마 《목욕탕집 남자들》(1995)에 이어 《웨딩드레스》, 《프로포즈》(1997), 《세상 끝까지》, 《미스터Q》(1998), 《토마토》(1999), 《해바라기》(1999) 등의 높은 시청률을 기록한 인기 드라마에서 연거푸 주연을 맡았고, 이외에도 수십 편의 광고 모델로 활약하며 명실상부 최고의 인기 톱스타로 자리매김했으며 그 당시 모든 패션과 유행의 아이콘이었다. 또한 김희선 황신혜의 뒤를 이은 1990년대를 대표하는 미인으로 불리며, 또한 당시 만 22세의 나이로 드라마 《미스터Q》로 SBS 연기대상에서 최연소의 나이에 연기대상을 수상하였고, 국내 외에도 드라마의 폭발적인 인기에 힘입어 중국에서도 대표적인 한류 스타로 발돋음하였다. 하지만 많은 인기를 끈 드라마와는 달리 영화에서는 《패자부활전》(1997년), 《카라》, 《자귀모》(1999년), 《비천무》(2000년)가 연이어 흥행참패를 당하면서 연기력 논란에 휩싸이기도 했다. 

Top-1 passage with distance 25.1361
엑스박스 라이브 아케이드는 마이크로소프트가 엑스박스 및 엑스박스 360 소유자에게 제공하는 온라인 서비스다. 팩맨 같은 고전 게임은 물론, 새로 나온 아케이드 게임도 제공한다. 라이브 아케이드에서는 다른 콘솔로 출시되었던 게임들도 제공하고 있는데, 플레이스테이션용으로 나왔던 《캐슬배니아: 밤의 교향곡》이 대표적이다. 이 서비스는 2004년 11월 3일에 공개되었다. 2005년 11월 22일에 엑스박스 360용으로 개편되었으며 현재는 대시보드와 통합되어 있다. 게임들은 캐주얼 게이머들에게 맞추어져 있다. 인기 있는 타이틀로는 《지오메트리 워즈 레트로 이볼브드》(Geometry Wars Retro Evolv